In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
sladb = config['db']['sladb']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [7]:
sql_count_sla_id = 'select count(*) from sladb.dbo.tbl_ref_sla'

In [8]:
sql_distinct_sla_id = 'select distinct sla_id from sladb.dbo.tbl_ref_sla'

In [10]:
sql_sla_code = '''select min(sla_code) as min_sla_code, 
                          max(sla_code) as max_sla_code, 
                          count(*) as n_sla_codes
                   from sladb.dbo.tbl_ref_sla_code'''

In [9]:
sql_date_category = 'select distinct(date_category_id) from sladb.dbo.tbl_ref_sla_season_category'

In [138]:
sql_sla_translation = 'select sla_id, sla_code, date_category_id from sladb.dbo.tbl_ref_sla_translation'

## Execute all of the sql queries and store the results

In [23]:
before_count_sla_id = pd.read_sql(con = engine, sql = sql_count_sla_id).values[0][0]

In [24]:
before_count_sla_id

6

In [25]:
before_squared_count_sla_id = before_count_sla_id * before_count_sla_id

In [26]:
before_squared_count_sla_id

36

In [29]:
before_distinct_sla_id = pd.read_sql(con = engine, sql = sql_distinct_sla_id)

In [33]:
before_distinct_sla_id

,sla_id
0,A
1,B
2,C
3,D
4,N
5,T


In [30]:
before_sla_code = pd.read_sql(con = engine, sql = sql_sla_code)

In [34]:
before_sla_code

,min_sla_code,max_sla_code,n_sla_codes
0,1,36,36


In [31]:
before_date_category = pd.read_sql(con = engine, sql = sql_date_category)

In [35]:
before_date_category

,date_category_id
0,1
1,2


In [32]:
before_sla_translation = pd.read_sql(con = engine, sql = sql_sla_translation)

In [36]:
before_sla_translation

,sla_trans_id,sla_id,date_category_id,sla_code
0,1,A,1,1
1,2,A,1,2
2,3,A,1,3
3,4,A,1,4
4,33,A,1,17
...,...,...,...,...
67,56,T,2,20
68,58,T,2,22
69,64,T,2,28
70,66,T,2,30


## Before Tests

In [188]:
timing = []

In [189]:
descriptions = []

In [190]:
results = []

### First before test

In [191]:
timing.append('before')

In [192]:
descriptions.append('''Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of 
                       sla_codes in tbl_ref_sla_code''')

In [193]:
results.append(before_squared_count_sla_id == before_sla_code['n_sla_codes'].values[0])

### Second before test

In [194]:
timing.append('before')

In [195]:
descriptions.append('''Test that the number of sla_codes in tbl_ref_sla_code is equal to the maximum sla_code
                       in tbl_ref_sla_code''')

In [196]:
results.append(before_sla_code['n_sla_codes'].values[0] == before_sla_code['max_sla_code'].values[0])

### Third before test

In [197]:
timing.append('before')

In [198]:
descriptions.append('''Test that the minimum sla_code in tbl_ref_sla_code is equal to 1''')

In [199]:
results.append(1 == before_sla_code['min_sla_code'].values[0])

### Fourth before test

In [200]:
timing.append('before')

In [201]:
descriptions.append('''Test that the count of records in tbl_ref_sla_translation is equal to the 
                       count of sla_codes in tbl_ref_sla_code multiplied by the number of date_category_ids 
                       in tbl_ref_sla_season_category''')

In [202]:
results.append(len(before_sla_translation) == (before_sla_code['n_sla_codes'].values[0] * len(before_date_category)))

### Fifth before test

In [203]:
timing.append('before')

In [204]:
descriptions.append('''Test the uniqueness of the rows in tbl_ref_sla_translation''')

In [205]:
before_sla_translation_nodup = before_sla_translation.drop_duplicates()

In [206]:
results.append(len(before_sla_translation) == len(before_sla_translation_nodup))

### Sixth before test

In [207]:
timing.append('before')

In [208]:
descriptions.append('''Test that all sla_ids, sla_codes and date_category_ids have been
                       accounted for in tbl_ref_sla_translation''')

In [209]:
unique_sla_translation = before_sla_translation.nunique()

In [210]:
results.append(((unique_sla_translation['sla_id'] == before_count_sla_id) 
                and (unique_sla_translation['date_category_id'] == len(before_date_category))
                and (unique_sla_translation['sla_code'] == before_sla_code['n_sla_codes'].values[0])))

In [211]:
list(zip(timing, descriptions, results))

[('before',
  'Check that the count of sla_ids (from tbl_ref_sla) squared is equal to the number of \n                       sla_codes in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the number of sla_codes in tbl_ref_sla_code is equal to the maximum sla_code\n                       in tbl_ref_sla_code',
  True),
 ('before',
  'Test that the minimum sla_code in tbl_ref_sla_code is equal to 1',
  True),
 ('before',
  'Test that the count of records in tbl_ref_sla_translation is equal to the \n                       count of sla_codes in tbl_ref_sla_code multiplied by the number of date_category_ids \n                       in tbl_ref_sla_season_category',
  True),
 ('before',
  'Test the uniqueness of the rows in tbl_ref_sla_translation',
  True),
 ('before',
  'Test that all sla_ids, sla_codes and date_category_ids have been\n                       accounted for in tbl_ref_sla_translation',
  True)]

### Insert the sla_id values into tbl_ref_sla so that the after tests can be conducted

In [ ]:
ninserts = 2

In [229]:
sla_id = [x for x in string.ascii_uppercase if x not in(before_distinct_sla_id.values)]

In [234]:
sla_inserts = [[x, 'Testing new SLA: '+ x, 1, 2] for x in sla_id]

In [235]:
sla_inserts_df = pd.DataFrame(sla_inserts[0:ninserts], 
                              columns = ['sla_id', 'sla_desc', 'sla_min_days', 'sla_max_days'])

In [ ]:
sla_insert_df.to_sql(con = engine, if_exists = 'append')

## After Tests